  Using cached langchain-1.2.9-py3-none-any.whl.metadata (5.7 kB)
  Using cached langchain_community-0.4.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached langchain_core-1.2.9-py3-none-any.whl.metadata (4.4 kB)
  Using cached langchain_google_genai-4.2.0-py3-none-any.whl.metadata (2.7 kB)
  Using cached langgraph-1.0.8-py3-none-any.whl.metadata (7.4 kB)
  Using cached pydantic-2.12.5-py3-none-any.whl.metadata (90 kB)
  Using cached langchain_classic-1.0.1-py3-none-any.whl.metadata (4.2 kB)
  Using cached sqlalchemy-2.0.46-cp313-cp313-macosx_11_0_arm64.whl.metadata (9.5 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached pyyaml-6.0.3-cp313-cp313-macosx_11_0_arm64.whl.metadata (2.4 kB)
  Using cached aiohttp-3.13.3-cp313-cp313-macosx_11_0_arm64.whl.metadata (8.1 kB)
  Using cached tenacity-9.1.3-py3-none-any.whl.metadata (1.2 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached pydantic_settings-2.12.0-py3-none-any.

In [ ]:
#import libraries

import os
import base64
from typing import List, Optional
import getpass
from langchain.chat_models import init_chat_model
from valyu import Valyu

from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents import create_openai_tools_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from pathlib import Path
from langchain_core.messages import HumanMessage
from langchain.agents import AgentExecutor
from prophet import Prophet
import yfinance as yf
import matplotlib.pyplot as plt
from typing import Annotated, Literal, TypedDict
import operator
from langchain_valyu import ValyuSearchTool
import datetime
from langchain_google_genai import ChatGoogleGenerativeAI



os.environ["OPENAI_API_KEY"] = "fill"
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = "fill"
os.environ["VALYU_API_KEY"] = "fill"
os.environ["GOOGLE_API_KEY"] = "fill"


model = ChatGoogleGenerativeAI(
    model="gemini-3-pro-preview",
    temperature=1.0,  
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)


ImportError: cannot import name 'create_openai_tools_agent' from 'langchain.agents' (/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/langchain/agents/__init__.py)

In [27]:


@tool
def mlModel(ticker: str):
    """ 
    Uses Facebook Prophet to predict stock prices 30 days ahead.
    Returns daily price targets and saves a plot to a .png file.
    the png files name is generated by graph_filename = f"{ticker}_forecast.png"
    """

    try:
        # search stock data

        data = yf.Ticker(ticker).history(period="2y")
        if data.empty:
            return "Error: No data found"
        
        # Format for prophet
        
        df = data.reset_index()
        df['ds'] = df['Date'].dt.tz_localize(None)
        df['y'] = df['Close']

        # Train

        m = Prophet(daily_seasonality=True)
        m.fit(df)

        # Predict

        future = m.make_future_dataframe(periods=30)
        forecast = m.predict(future)

        # Save the Graph

        fig1 = m.plot(forecast)
        plt.title(f"{ticker} 30-Day Forecast")
        plt.xlabel("Date")
        plt.ylabel("Price")

        graph_filename = f"{ticker}_forecast.png"
        plt.savefig(graph_filename)
        plt.close()

        # List 30 day prediction
        future_data = forecast.tail(30)

        daily_tracking = []
        for index, row in future_data.iterrows():
            date_str = row['ds'].strftime('%Y-%m-%d')
            price_str = f"${row['yhat']:.2f}"
            daily_tracking.append(f"{date_str}: {price_str}")

        daily_summary = "\n".join(daily_tracking)

        latest_pred = forecast.iloc[-1]['yhat']
        current_price = df.iloc[-1]['y']
        trend = "UP" if latest_pred > current_price else "DOWN"
        
        # Return Everything
        return (f"Analysis Complete for {ticker}\n"
                f"Graph saved to: {graph_filename}\n"
                f"Trend: {trend}\n\n"
                f"Daily Price Targets (Next 30 Days):\n"
                f"{daily_summary}")
    
    except Exception as e:
        return f"Prediction failed: {e}"
    
@tool 
def brownianModel(TICKER, START_DATE, END_DATE, PRED_END_DATE):
    """ 
    Brownian model for stocks
    START_DATE: from which (past) date should we begin considering the Monte Carlo.
    END_DATE: from which (past) date are we ending the considering of its data (presumably as late as possible, given data is available)
    PRED_END_DATE: the (future) date which the client is willing to predict until.  
    """
    # main variables
    stock_name = TICKER
    start_date = START_DATE
    end_date = END_DATE
    pred_end_date = PRED_END_DATE
    scen_size = 10000

    # -----------------------------
    # Download and prepare data
    # -----------------------------

    prices = yf.download(tickers=stock_name, start=start_date, end=pred_end_date)
    prices = prices['Close']

    # ---- FIX 1: Robust price column selection ----
    #if isinstance(prices.columns, pd.MultiIndex):
        #if ('Adj Close', stock_name) in prices.columns:
            #prices = prices[('Adj Close', stock_name)]
       #else:
            #prices = prices[('Close', stock_name)]
    #else:
       # if 'Adj Close' in prices.columns:
            #prices = prices['Adj Close']
        #else:
            #prices = prices['Close']

    
    # Generate business days (weekdays only)
    future_dates = pd.bdate_range(start=pd.to_datetime(end_date) + pd.Timedelta(days=1),
                    end=pd.to_datetime(pred_end_date))

   
    train_set = prices.loc[:end_date]
    # Create DataFrame with a 'Date' column
    dframe = pd.DataFrame({'Prediction Date': future_dates})
    
    daily_returns = ((train_set / train_set.shift(1)) - 1)[1:]



    So = train_set.iloc[-1]
    dt = 1  # day

    n_of_wkdays = pd.date_range(
        start=pd.to_datetime(end_date) + pd.Timedelta('1 days'),
        end=pd.to_datetime(pred_end_date)
    ).to_series().map(lambda x: 1 if x.isoweekday() in range(1, 6) else 0).sum()

    T = n_of_wkdays
    N = int(T / dt)
    t = np.arange(1, N + 1)

    mu = np.mean(daily_returns)
    sigma = np.std(daily_returns)

    b = {str(scen): np.random.normal(0, 1, N) for scen in range(1, scen_size + 1)}
    W = {str(scen): b[str(scen)].cumsum() for scen in range(1, scen_size + 1)}

    drift = (mu - 0.5 * sigma ** 2) * t
    diffusion = {str(scen): sigma * W[str(scen)] for scen in range(1, scen_size + 1)}

    S = np.array([So * np.exp(drift + diffusion[str(scen)]) for scen in range(1, scen_size + 1)])
    S = np.hstack((np.array([[So] for _ in range(scen_size)]), S))


    S_max = [S[:, i].max() for i in range(0, N)]
    S_min = [S[:, i].min() for i in range(0, N)]
    S_pred = 0.5 * np.array(S_max) + 0.5 * np.array(S_min)

    # Standard Monte Carlo estimator: expected price (mean across simulations)
    #S_pred = np.median(S, axis=0)[1:]


    # Align prediction length with available real prices
    min_len = min(len(dframe['Prediction Date']), len(S_pred))

    final_df = pd.DataFrame({
        'pred': S_pred[:min_len]
    }, index=dframe['Prediction Date'])

    #mse = np.mean((final_df['pred'] - final_df['real']) ** 2)
    # Convert to string row by row as "Date: Price"

    result = '\n'.join(f"{date.date()}: {price}" for date, price in zip(final_df.index, final_df['pred']))
    return result


@tool
def valyu_search_tool(query: str):
    """
    Searches the web using Valyu API to get specific market news and context.
    """
    try:
        # Using the .answer method from your provided code
        response = valyu.answer(query)
        
        # Extract content safely
        if hasattr(response, 'contents'):
            return response.contents
        elif isinstance(response, dict) and 'contents' in response:
            return response['contents']
        else:
            return str(response)
            
    except Exception as e:
        return f"Search failed: {e}"
    







In [28]:
trend_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a Quantitative Analyst. Use the provided ML and Statistical tools to analyze the stock ticker provided. Summarize the technical outlook."),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])
trend_agent = create_openai_tools_agent(model, [mlModel, brownianModel], trend_prompt)
trend_executor = AgentExecutor(agent=trend_agent, tools=[mlModel, brownianModel], verbose=True)

noise_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a Market Researcher. Use the search tool to find recent news, sentiment, and macro factors affecting the stock."),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])
noise_agent = create_openai_tools_agent(model, [valyu_search_tool], noise_prompt)
noise_executor = AgentExecutor(agent=noise_agent, tools=[valyu_search_tool], verbose=True)

In [29]:
from pydantic import BaseModel, Field
from langgraph.types import Send
from langgraph.graph import StateGraph, START, END



class AgentState(TypedDict):
    query: str
    results: Annotated[List[str], operator.add] 

def run_trend_agent(state: AgentState):
    """Invokes the Quant Agent"""
    print("Executing Trend Agent")
    response = trend_executor.invoke({"input": f"Analyze {state['query']}"})
    return {"results": [f"QUANT ANALYSIS:\n{response['output']}"]}

def run_noise_agent(state: AgentState):
    """Invokes the Research Agent"""
    print("Executing Noise Agent")
    response = noise_executor.invoke({"input": f"Find news for {state['query']}"})
    return {"results": [f"RESEARCH ANALYSIS:\n{response['output']}"]}

def aggregator(state: AgentState):
    """Combines results into a final answer"""
    print("Aggregating Results")
    final_prompt = (
        f"Combine the following reports into a comprehensive investment memo for {state['query']}:\n\n"
        + "\n\n".join(state["results"])
    )
    response = model.invoke(final_prompt)
    print(f"\nFINAL ANSWER:\n{response.content}")
    return {"results": [response.content]}

class RouteSchema(BaseModel):
    targets: List[Literal["quant", "research"]] = Field(
        description="Which agents to hire? Quant for numbers/charts, Research for news/sentiment."
    )

workflow = StateGraph(AgentState)


workflow.add_node("trend_node", run_trend_agent)
workflow.add_node("noise_node", run_noise_agent)
workflow.add_node("aggregator", aggregator)

def route_query(state: AgentState) -> List[Send]:
    structured_llm = model.with_structured_output(RouteSchema)
    decision = structured_llm.invoke(f"Analyze: {state['query']}")
    
    routes = []
    if "quant" in decision.targets:
        routes.append(Send("trend_node", state))
    if "research" in decision.targets:
        routes.append(Send("noise_node", state))
    if not routes:
        routes = [Send("trend_node", state), Send("noise_node", state)]
        
    return routes

workflow.add_conditional_edges(START, route_query)
workflow.add_edge("trend_node", "aggregator")
workflow.add_edge("noise_node", "aggregator")
workflow.add_edge("aggregator", END)

app = workflow.compile()



if __name__ == "__main__":
    user_query = "What is the outlook for AAPL stock?"
    inputs = {"query": user_query, "results": []}

    for output in app.stream(inputs):
        pass 









RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}